# Setup

In [ ]:
%load_ext autoreload
%autoreload 2
# %matplotlib widget
%pdb off

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import plotly
from IPython.display import display, HTML
import ast
from scipy.optimize import minimize
import seaborn as sns
from functools import wraps

plotly.offline.init_notebook_mode()
display(HTML(
    '<script type="text/javascript" async src="https://cdnjs.cloudflare.com/ajax/libs/mathjax/2.7.1/MathJax.js?config=TeX-MML-AM_SVG"></script>'
))

plt.rcParams['figure.dpi'] = 140
im_scaling = .75
plt.rcParams['figure.figsize'] = [6.4 * im_scaling, 4.8 * im_scaling]

home_dir = "./"
display(home_dir)
plt.close('all')

# Stuff Rob Should Not Need to Care About

## Read in results

In [ ]:
multiRun_dir = f"{home_dir}/CHARLES/multiRuns/"
plotFolder = f"{multiRun_dir}"

In [ ]:
def evalStringAsList(s):
    if isinstance(s, str) and s[0] == "[" and s[-1] == "]":
        return ast.literal_eval(s)
    else:
        return s


def dict_apply(func):
    @wraps(func)
    def wrapper(d):
        if isinstance(d, dict):
            return {k: func(v) for k, v in d.items()}
        else:
            return func(d)
    return wrapper

In [ ]:
overWrite = False
pcStatsMI = pd.read_csv(f"{multiRun_dir}/pointCloudStatsNoIntMI.csv", index_col=[0,1])
flowStatsMI = pd.read_csv(f"{multiRun_dir}/flowStatsMI.csv", index_col=[0,1])
roomVentilationMI = pd.read_csv(f"{multiRun_dir}/roomVentilationMI.csv", index_col=[0,1])
flowStatsMI = flowStatsMI.map(evalStringAsList)
roomVentilationMI = roomVentilationMI.map(evalStringAsList)
flowStatsMI["pNoInt"] = np.nan
flowStatsMI.update(pcStatsMI)

## Helper Utils

In [ ]:
g = 10
beta = 0.0034
rho = 1.225
hm = 6
window_dim = hm/2/4
A = window_dim ** 2

def getWindBuoyantP(rho, flowParams):
    p_w = flowParams["p_w"]
    z = flowParams["z"]
    delT = flowParams["delT"]
    delrho = -rho * beta * delT
    return (delrho * g * z) + p_w # delP is outdoor minus indoor, while p0/rho is indoor minus outdoor, driving positive flow into the room (oppiste textbook)

def flowFromP(rho, C_d, A, delp):
    delp=np.array(delp)
    S = np.zeros_like(delp)
    S[delp!=0] = abs(delp[delp!=0]) / delp[delp!=0]
    return S * C_d * A * np.sqrt(2 * abs(delp) / rho)

def CFromFlow(rho, q, A, delp):
    delp=np.array(delp)
    S = np.zeros_like(delp)
    S[delp!=0] = abs(delp[delp!=0]) / delp[delp!=0]
    C = q /  (S * A * np.sqrt(2 * abs(delp) / rho))
    return C

def flowField(p_0, rho, flowParams):
    C_d = flowParams["C_d"]
    A = flowParams["A"]
    rooms = flowParams["rooms"]
    delP = -np.matmul(rooms, p_0) + getWindBuoyantP(rho, flowParams) 
    return flowFromP(rho, C_d, A, delP)

def getC(p_0, rho, flowParams):
    A = flowParams["A"]
    q = flowParams["q"]
    rooms = flowParams["rooms"]
    delP = -np.matmul(rooms, p_0) + getWindBuoyantP(rho, flowParams)
    return CFromFlow(rho, q, A, delP)

def qObjective(p_0, rho, flowParams):
    qs = flowField(p_0, rho, flowParams)
    rooms = flowParams["rooms"]
    qRooms = np.matmul(rooms.T, qs)
    return np.sum(qRooms**2)

def findOptimalP0(rho, flowParams):
    bounds = np.array([np.min(getWindBuoyantP(rho, flowParams)), np.max(getWindBuoyantP(rho, flowParams))])
    x0 = np.mean(bounds)
    NRooms = flowParams["rooms"].shape[1]
    bounds = np.tile(bounds, (NRooms, 1))
    x0 = np.tile(x0, NRooms)
    return minimize(qObjective, x0=x0, bounds=bounds, args=(rho, flowParams))

## Primary Ventilation Calcs


In [ ]:

def getC_ds_All(aCorner, aCross, aDual, aSingle, aSinA, aCosA,  aShear, aNormal, aWS, AofA = 180, shear=0, normal=0, WS=0, roomType="corner"):
    AofA *= np.pi / 180
    C_d = aSinA * np.sin(AofA) + aCosA * np.cos(AofA) + aShear * shear + aNormal * normal + aWS * WS
    if roomType == "corner":
        return C_d + aCorner
    if roomType == "cross":
        return C_d + aCross
    if roomType == "dual":
        return C_d + aDual
    if roomType == "single":
        return C_d + aSingle

def getC_ds(params, typeC_d="AofA", AofA=0, shear=0, normal=0, openingType="xwindow", roomType="corner"):
    if typeC_d == "All":
        return getC_ds_All(*params, AofA=AofA, shear=shear, normal=normal, WS=1, roomType=roomType)
    else:
        raise Exception(f"Unrecognized typeC_d {typeC_d}")



def update_flow_and_ventilation(flowStatsMI, roomVentilationMI, paramsC_d, useDoors=True, pTypes = {"pNoInt": "pNoInt"}, typeC_d="AofA"):
    flowStatsMI["cosAofA"] = np.round(np.cos(flowStatsMI["AofA"] * np.pi / 180), 2)
    flowStatsMI["sinAofA"] = np.round(np.sin(flowStatsMI["AofA"] * np.pi / 180), 2)

    for pType in pTypes:
        roomVentilationMI[f"{pType}-success"] = False
    for (run, room), row in roomVentilationMI.iterrows():
        flowParams = {
            "C_d": [],
            "A": [],
            "z": [],
            "delT": [],
            "q": [],
            "rooms": [],
        }
        for pType in pTypes:
            flowParams[pType] = []
        for windowKey in row["windowKeys"]:
            for pType, pCol in pTypes.items():
                flowParams[pType].append(flowStatsMI.loc[(run, windowKey), pCol])
            C_d = getC_ds(
                paramsC_d,
                typeC_d = typeC_d,
                AofA=flowStatsMI.loc[(run, windowKey),"AofA"], 
                shear=flowStatsMI.loc[(run, windowKey),"EP_shear"], 
                normal=flowStatsMI.loc[(run, windowKey),"EP_normal"],
                openingType = flowStatsMI.loc[(run, windowKey),"openingType"],
                roomType = row["roomType"],
                )
            flowParams["C_d"].append(C_d)
            flowParams["A"].append(A)
            flowParams["z"].append(flowStatsMI.loc[(run, windowKey), "y"])  # y is vertical in simulation
            flowParams["delT"].append(row["mean-T-room"])
            flowParams["q"].append(flowStatsMI.loc[(run, windowKey), "mean-mass_flux"])
            if "dual" in room and useDoors:
                roomCord = windowKey.split("_")[1]
                if roomCord == "0-1":
                    roomRow = [1, 0]
                elif roomCord == "1-1":
                    roomRow = [0, 1]
                else:
                    raise Exception(f"Unrecognized room {roomCord} in dual room")
            else:
                roomRow = [1]
            flowParams["rooms"].append(roomRow)

        if "dual" in room and useDoors:
            H = 3
            for pType in pTypes:
                flowParams[pType].append(0)
            flowParams["C_d"].append(1)
            flowParams["A"].append(A * 3)
            flowParams["z"].append(H / 2)
            flowParams["delT"].append(row["mean-T-room"])
            qRooms = np.matmul(np.array(flowParams["rooms"]).T, np.array(flowParams["q"]))
            flowParams["q"].append(np.diff(qRooms)[0])
            flowParams["rooms"].append([1, -1])

        flowParams = dict_apply(np.array)(flowParams)

        sinAofAs = []
        cosAofAs = []
        for i, windowKey in enumerate(row["windowKeys"]):
            sinAofAs.append(flowStatsMI.loc[(run, windowKey), "sinAofA"])
            cosAofAs.append(flowStatsMI.loc[(run, windowKey), "cosAofA"])

        roomVentilationMI.loc[(run, room), "sinAofA"] = np.mean(sinAofAs)
        roomVentilationMI.loc[(run, room), "cosAofA"] = np.mean(cosAofAs)

        for pType in pTypes:
            NRooms = flowParams["rooms"].shape[1]
            flowParams["p_w"] = flowParams[pType]
            p0_meas = [row["mean-p-room"] for i in range(NRooms)]
            C_ds = getC(np.array(p0_meas), rho, flowParams)

            for i, windowKey in enumerate(row["windowKeys"]):
                flowStatsMI.loc[(run, windowKey), f"{pType}-C_d"] = C_ds[i]

            optResults = findOptimalP0(rho, flowParams)

            p0 = optResults.x
            roomVentilationMI.loc[(run, room), f"{pType}-p0"] = np.mean(p0)
            roomVentilationMI.loc[(run, room), f"{pType}-success"] = optResults.success
            qs = flowField(np.array(p0), rho, flowParams)

            for i, windowKey in enumerate(row["windowKeys"]):
                flowStatsMI.loc[(run, windowKey), f"{pType}-q_model"] = qs[i]
                flowStatsMI.loc[(run, windowKey), f"{pType}-netq_model"] = abs(qs[i])

            if "dual" in room and useDoors:
                qs = qs[0:-1]
            roomVentilationMI.loc[(run, room), f"{pType}-q_model"] = np.sum(abs(np.array(qs))) / 2

    return flowStatsMI, roomVentilationMI


# For Rob


## The One True f(x)

In [ ]:
def fRob(x, theta):
    """
    Simplified function for rob to demonstrate emulator
    x: (flowStatsMI, roomVentilationMI})
    theta: [aCorner, aCross, aDual, aSingle, aSinA, aCosA,  aShear, aNormal, aWS]
            - These are the parameters corresponding to getC_ds_All (meaning all parameters are used). They are currently liner scaling but wouldn't have to be.
    """
    flowStatsMI, roomVentilationMI = x
    flowStatsMI, roomVentilationMI = update_flow_and_ventilation(flowStatsMI, roomVentilationMI, theta, typeC_d="All")
    y = (flowStatsMI, roomVentilationMI)
    return y

### Calling f()

In [ ]:
x = (flowStatsMI, roomVentilationMI)
theta = [1, 1, 1, 1, 0, 0, 0, 0, 0]
y = fRob(x, theta)
flowStatsMI, roomVentilationMI = y

### Dropping Skylight Rooms with Poor Data

In [ ]:
# drop rooms with skylights
flowStatsMINoSL = flowStatsMI[flowStatsMI["AofA"] % 1 == 0].copy()
roomVentilationMINoSL = roomVentilationMI[roomVentilationMI["sinAofA"].notna()].copy()

### Loss Function

In [ ]:
def qRMSE(df):
    return np.sqrt(np.mean((df["pNoInt-q_model"] - df["mean-mass_flux"])**2))

display(f"the RMSE for room ventilation is {qRMSE(roomVentilationMINoSL)}")

### Linear C_d fits and plot

In [ ]:
dfFit = flowStatsMINoSL.dropna(subset=["pNoInt-q_model"])
# Reshape x into a 2D column vector
x = dfFit["pNoInt-q_model"].values.reshape(-1, 1)  # Ensure it's 2D
y = dfFit["mean-mass_flux"].values  # Keep y as 1D

# Solve for the slope (forcing intercept = 0)
CfitWindow, _, _, _ = np.linalg.lstsq(x, y, rcond=None)
CfitWindow = CfitWindow[0]

# Print the fitted coefficient
print("Fitted Coefficient:", CfitWindow)

# Fit C for each room type separately
room_types = dfFit['roomType'].unique()
CfitRoomTypes = {}

for room_type in room_types:
    dfRoomType = dfFit[dfFit['roomType'] == room_type]
    x = dfRoomType["pNoInt-q_model"].values.reshape(-1, 1)  # Ensure it's 2D
    y = dfRoomType["mean-mass_flux"].values  # Keep y as 1D

    # Solve for the slope (forcing intercept = 0)
    Cfit, _, _, _ = np.linalg.lstsq(x, y, rcond=None)
    CfitRoomTypes[room_type] = Cfit[0]

# Print the fitted coefficients for each room type
for room_type, Cfit in CfitRoomTypes.items():
    print(f"Fitted Coefficient for {room_type}: {Cfit}")



In [ ]:
# Create the scatter plot
plt.figure(figsize=(10, 6))
sns.scatterplot(data=flowStatsMI, x='pNoInt-q_model', y="mean-mass_flux", hue="roomType", style="roomType")

# Overlay the lines from the fitted C values
x_vals = np.linspace(flowStatsMI["mean-mass_flux"].min(), flowStatsMI["mean-mass_flux"].max(), 100)

# Overall fitted line
y_vals_overall = x_vals * CfitWindow
plt.plot(x_vals, y_vals_overall, label='Overall Fit', color='black')

# Fitted lines for each room type
for room_type, Cfit in CfitRoomTypes.items():
    y_vals_room = Cfit * x_vals
    plt.plot(x_vals, y_vals_room, label=f'{room_type} Fit')

plt.legend()
plt.xlabel('Mean Mass Flux')
plt.ylabel('pNoInt-q_model')
plt.title('Scatter Plot with Fitted Lines')